## Portfolio Data Builder

This notebook contains the code used to develop the simple moving averages trading algorithm. After developing the required algorithm, the data is exported to 4 csv files; 

- profitable_stock.csv - profits for all 500 stocks in the SP 500
- top_profitable_stock.csv - top 10 profitable stocks
- top_prof_stock_prices.csv - top 10 profitable stocks with closing prices - SP index appended
- sample_moving_amzn.csv - sample Amazon file closing prices with 50 and 100 moving avg appended

that are used to conduct a data analysis and visualizations in the *insert notebook name here* notebook.

yFinance API was attempted but during initial data pulls, blank / empty data rows were found that were affecting results.  Switch was made to use the Yahoo finance library

In [12]:
# Initial imports
import os
import requests
import datetime
import calendar
import json
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import plotly.express as px
import pandas_datareader.data as web
%matplotlib inline

In [14]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [15]:
# Import hvplot.pandas after pn.extension
# This avoids plotly initialization failure
import hvplot.pandas

## Notebook Functions

In [16]:
def generate_stock_prices(stock):

    """
    Funtion returns dataframe containing the closing prices and date for supplied stock symbol between
    July 26, 2015 and July 24, 2020.
    """
    
    # Create empty dataframe for closing prices
    data = []
    
    # Pull data for specified stock symbol and dates
    stock_prices = pd.DataFrame(columns = ["date", "close"])
    data = web.DataReader(stock, "yahoo","2015-07-26","2020-07-24" )
        
    # drop all rows with any NaN and NaT values
    data = data.dropna()
    
    data = data.reset_index()
    data = data.sort_values(by = ['Date'], ascending=True)
    
    #Loop through stock data and pull only date and close price
    for key, value in data.iterrows():
        
        if value.empty:
            print(key)
        
        stock_prices = stock_prices.append({'date': value['Date'], 
                                'close': round(value['Close'],2)}, 
                                    ignore_index=True)
    return stock_prices



def calculate_stock_profit(dataframe):

    """
    Funtion appends the 50 and 100 moving average to the supplied stock dataframe.  Determines the when the averages cross calculating
    when to buy and sell.  Profit is cummulated based on an intial investment of 1000.00
    """
    
    day_num=0
    curr_pos=0
    curr_bal=0
    profit=0
    acum_profit=0
    
    stock_profit = dataframe.set_index(dataframe['date'])  # drop = true

    stock_profit['100ma'] = stock_profit['close'].rolling(window=100,min_periods=0).mean()
    stock_profit['50ma'] = stock_profit['close'].rolling(window=50,min_periods=0).mean()


    for day_num in range(len(stock_profit)):
        if day_num>0:
            ma_100 = round(stock_profit.iloc[day_num]['100ma'],2)
            ma_50 = round(stock_profit.iloc[day_num]['50ma'],2)
            pre_ma_100 = round(stock_profit.iloc[day_num-1]['100ma'],2)
            pre_ma_50 = round(stock_profit.iloc[day_num-1]['50ma'],2)


            trade=trader(pre_ma_100,pre_ma_50,ma_100,ma_50)


            if trade=="BUY": 
                
                curr_pos = curr_pos+1000
                curr_bal = curr_pos*stock_profit.iloc[day_num]['close']
                
            elif trade=="SELL":    
               
                new_bal = curr_pos*stock_profit.iloc[day_num]['close']
                profit = +new_bal-curr_bal
                acum_profit = acum_profit+profit
                curr_pos = curr_pos-1000
                curr_bal = 0
               

    return acum_profit




def trader(pre_ma_100, pre_ma_50, ma_100, ma_50):

    """
    Funtion determines the when the averages cross calculating and returns a 
    when to buy or sell flag.  
    """

    if(pre_ma_50<pre_ma_100 and ma_50>ma_100):        
        order="BUY"
    elif(pre_ma_50>pre_ma_100 and ma_50<ma_100):        
        order="SELL"
    else: order="Do Nothing"
    
    return order

### Profitable Stock Builder

In [26]:
# Setup input and output files and locations
filepath = Path("resources/stock_symbols.csv")
output = Path("resources/profitable_stock.csv")

stock_df = pd.read_csv(filepath)

#Define profitable stock dataframe
profitable_stock_df = pd.DataFrame(columns = ["symbol", "profit", "sector"])

# Loop through S&P 500 stock symbols
for key, value in stock_df.iterrows():

    print(value['Symbol'])
    
    #Pull closing prices for each stock in S&P 500 symbols 
    stock_df = generate_stock_prices(value['Symbol'])
    
    # Call function to calculate stock profitability
    profitable_stock_df = profitable_stock_df.append({'symbol': value['Symbol'],
                                                      'profit': round(calculate_stock_profit(stock_df),2),
                                                      'sector': value['Sector']},ignore_index=True)
# Output profitable stocks to csv file
profitable_stock_df.to_csv(output)

profitable_stock_df.head()

MMM
AOS
ABT
ABBV
ABMD
ACN
ATVI
ADBE
AAP
AMD
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
ANTM
AON
APA
AIV
AAPL
AMAT
APTV
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
BKR
BLL
BAC
BAX
BDX
BBY
BIO
BIIB
BLK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
CHRW
COG
CDNS
CPB
COF
CAH
KMX
CCL
CARR
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CTXS
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
CPRT
GLW
CTVA
COST
COTY
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DRE
DD
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EOG
EFX
EQIX
EQR
ESS
EL
RE
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FRC
FE
FISV
FLT
FLIR
FLS
FMC
F
FTNT
FTV
FBHS
FOXA
FOX
BEN
FCX
GPS
GRMN
IT
GD
GE
GIS
GM
GPC
GILD
GPN
GL
GS
GWW
HRB
HAL
HBI
HIG
HAS
HCA
PEAK
HSIC
HES
HPE
HLT
HFC
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUM
HBAN
HII
IEX
IDXX
INFO
ITW

,symbol,profit,sector
0,MMM,-7670.0,Industrials
1,AOS,6450.0,Industrials
2,ABT,4630.0,Health Care
3,ABBV,59140.0,Health Care
4,ABMD,-306210.0,Health Care


### Generate top 10 profitable from the SP 500 stocks

In [27]:
# Setup inputoutput files and locations
output = Path("resources/top_profitable_stock.csv")

# Determine top 10 profitable stocks
top_profitable_stock_df = profitable_stock_df.sort_values(by = ['profit'], ascending=False).head(10)

# Output top 10 to file
top_profitable_stock_df.to_csv(output)

### Capture closing prices for top 10 stocks from SP 500

In [29]:
# Setup inputoutput files and locations
output = Path("resources/top_prof_stock_prices.csv")

# Setup prices dataframe
top_stock_prices_df = pd.DataFrame()

# Add S&P 500 index symbol to list for analysis
top_prof_stock_df = top_profitable_stock_df.append({'symbol': 'SPY',
                                                      'profit': 0.0,
                                                      'sector': 'Index'},ignore_index=True)
# Loop through top 10 stock symbols
for key, value in top_prof_stock_df.iterrows():

    print(value['symbol'])
    
    # Pull closing prices for each top 10 stock symbols
    df = generate_stock_prices(value['symbol'])
    
    # Setup up dataframe with close price column, set date as index
    df = df.rename(columns ={"close": value['symbol']})
    df = df.set_index(df['date'])
    df = df.drop(columns=['date'])
    
    # Check if the dataframe is empty and set up append
    if top_stock_prices_df.shape[0] == True:
        top_stock_prices_df = df
    else:
        # Append price dataframe to total dataframe
        top_stock_prices_df = pd.concat([df,top_stock_prices_df],axis="columns")

# Output close prices to file
top_stock_prices_df.to_csv(output)

top_stock_prices_df.head()

AMZN
BIO
NOW
MSCI
IPGP
SPGI
NFLX
TDY
PNC
MKTX
SPY


,SPY,MKTX,PNC,TDY,NFLX,SPGI,IPGP,MSCI,NOW,BIO,AMZN
date,,,,,,,,,,,
2015-07-27,206.79,97.95,97.51,102.67,106.43,99.59,79.86,65.73,76.34,146.45,531.41
2015-07-28,209.33,97.66,98.14,103.67,106.90,98.40,89.71,66.00,78.63,149.45,526.03
2015-07-29,210.77,98.16,98.55,104.99,107.08,101.48,93.30,66.91,77.98,150.46,529.00
2015-07-30,210.82,97.07,98.67,103.70,111.56,102.84,92.56,67.97,81.03,150.81,536.76
2015-07-31,210.50,97.80,98.18,103.67,114.31,101.75,92.22,68.16,80.50,150.74,536.15


### Create 50 and 100 moving avg dataframe with close prices

In [33]:
# Create sample csv file with prices and moving average prices to generate sample graph

# Setup inputoutput files and locations
output = Path("resources/sample_moving_amzn.csv")


# Pull closing prices stock AMZN
sample_moving_price = generate_stock_prices('AMZN')

# Set date as index
sample_moving_result = sample_moving_price.set_index(sample_moving_price['date'])  # drop = true
sample_moving_result = sample_moving_result.drop(columns=['date'])

# Append
sample_moving_result['50ma'] = sample_moving_result['close'].rolling(window=50,min_periods=0).mean()
sample_moving_result['100ma'] = sample_moving_result['close'].rolling(window=100,min_periods=0).mean()

# Output close prices to file
sample_moving_result.to_csv(output)

sample_moving_result.head()

,close,50ma,100ma
date,,,
2015-07-27,531.41,531.410000,531.410000
2015-07-28,526.03,528.720000,528.720000
2015-07-29,529.00,528.813333,528.813333
2015-07-30,536.76,530.800000,530.800000
2015-07-31,536.15,531.870000,531.870000


## Data Clean and Analysis

Prior to pulling data from yahoo, each stocks dataframe was checked for nulls and they were removed.  Although we are looking for the top profitable stock from the SP 500 following box plot shows the profit distribution of all 500 stocks.

In [31]:
#Generate box plot of all 500 stocks
path = Path("resources/profitable_stock.csv")

profitable_portfolio_df = pd.read_csv(path, index_col=0)

all_profit_plot = px.box(profitable_portfolio_df["profit"], orientation='h', points='all', 
                                    title="Distribution of SP 500 Stock Profit",
                                    labels = {'variable':'Stock','value':'Profit'})

profit_plot = px.box(profitable_portfolio_df["profit"], orientation='h', points='all', 
                                    title="Distribution of SP 500 Stock Profit/Loss",
                                    labels = {'variable':'Stock','value':'Profit'},
                                    color_discrete_sequence=["red", "green"],
                                    color=(profitable_portfolio_df['profit'] > 0).map({False: 'Loss',
                                                    True: 'Gain'}),)
all_profit_plot.show()
profit_plot.show()